**Querying postgreSQL in Jupyter notebook**

Useful for writing notes and iterating over SQL queries. You can look at the "hard" examples down below to show how queries can be broken down into smaller parts and then combined into a more complicated query.
-Ben

# Setup and custom tables

In [1]:
import pandas as pd
import sqlalchemy
import sqlalchemy_utils
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [2]:
# Define a database name
# Set your postgres username
dbname = "baseball"
username = "lacar"  # change this to your username

# Working with PostgreSQL in Python
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database=dbname, user=username)

# Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine("postgres://%s@localhost/%s" % (username, dbname))
print(engine.url)

postgres://lacar@localhost/baseball


## Using CTE

In [3]:
# Just use a date difference less than 30 days

sql_query = """
WITH posts (user_id, post, time)
AS (VALUES
(1, 'A', CAST('2-14-20' AS date)),
(2, 'B', CAST('2-14-20' AS date)),
(3, 'C', CAST('2-15-20' AS date)),
(1, 'B', CAST('2-15-20' AS date)),
(2, 'A', CAST('2-16-20' AS date)),
(3, 'B', CAST('2-17-20' AS date)),
(1, 'D', CAST('2-18-20' AS date)))

SELECT *
FROM posts
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

,user_id,post,time
0,1,A,2020-02-14
1,2,B,2020-02-14
2,3,C,2020-02-15
3,1,B,2020-02-15
4,2,A,2020-02-16
5,3,B,2020-02-17
6,1,D,2020-02-18


## Using pandas

### Dates

In [197]:
# Generate random date ranges
# From https://towardsdatascience.com/mastering-dates-and-timestamps-in-pandas-and-python-in-general-5b8c6edcc50c

import random
import time
from dateutil.parser import parse
def str_time_prop(start, end, format, prop):
    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(format, time.localtime(ptime))

selected_format = '%Y-%m-%d %H:%M:%S'

def random_date(start, end, prop):
    return parse(str_time_prop(start, end, selected_format, prop)).strftime(selected_format)

def make_date(begin_dt, end_dt):
    return random_date(begin_dt, end_dt, random.random())

# e.g. make_date("2020-01-01 13:40:00", "2020-01-14 14:50:00")


# Generate dates (my function)
def generate_dates(n_dates, begin_dt, end_dt):
    return sorted([make_date(begin_dt, end_dt) for x in range(n_dates)])

### Names

In [5]:
# Generate names (my function)
def generate_name_list(n_names):
    import names   # needed to pip install
    name_list = list()
    for i in range(n_names):
        name_list.append(names.get_first_name())
    return name_list

### Random values within a range

In [190]:
def generate_values(n_vals, lowest, highest):
    import random
    random_vals = random.sample(range(lowest, highest), n_vals)
    return random_vals

In [ ]:
# Also try random.random()

In [191]:
generate_values(10, 1, 20)

[10, 4, 17, 1, 6, 13, 16, 2, 11, 12]

### Multipurpose 3-digit codes

In [6]:
# Generate 3-digit codes (e.g. city ids) (my function)
def generate_codes(n_codes):
    # 3 digits between 110 and 999 without repeating
    import random
    code_ids = random.sample(range(110, 1000), n_codes)
    return code_ids

### Multipurpose custom values

In [7]:
# Generate random list following input of a set of values to choose
def generate_custom_vals(list2consider, n_items):
    custom_list = np.random.choice(list2consider, size=n_items, replace=True).tolist()
    return custom_list

### Example

In [199]:
my_date_list = generate_dates(10, "2020-01-01 13:40:00", "2020-01-14 14:50:00")
my_name_list = generate_name_list(10)
my_city_codes = generate_codes(10)

# Custom list 1
my_list2consider = ['desktop-browser','mobile-browser','ios-native','android-native']
my_list2consider4table = generate_custom_vals(my_list2consider, 10)

# Custom list 2
my_list2consider = ['US', 'Canada', 'Mexico']
my_countries4table = generate_custom_vals(my_list2consider, 10)

In [200]:
col_1 = range(1, 11)
col_2 = pd.to_datetime(my_date_list)
col_3 = my_list2consider4table
col_4 = my_countries4table
table1 = pd.DataFrame([col_1, col_2, col_3, col_4]).T
table1.columns = ['user_id', 'join_ts', 'join_client', 'country']

table1

# Temp table created here that I'll just over-write with each new problem
# table1.to_sql('user_summary', engine, if_exists='replace')


,user_id,join_ts,join_client,country
0,1,2020-01-03 04:44:56,mobile-browser,Mexico
1,2,2020-01-05 02:11:16,desktop-browser,Mexico
2,3,2020-01-07 00:39:42,ios-native,US
3,4,2020-01-08 04:01:35,mobile-browser,Canada
4,5,2020-01-10 06:32:34,desktop-browser,Canada
5,6,2020-01-11 19:32:28,android-native,Canada
6,7,2020-01-11 19:35:18,ios-native,Canada
7,8,2020-01-12 15:21:11,android-native,US
8,9,2020-01-13 14:44:13,ios-native,Canada
9,10,2020-01-14 07:19:43,desktop-browser,Canada


# QotD 5/5/20

In [27]:
# Just use a date difference less than 30 days

sql_query = """
WITH posts (user_id, post, time)
AS (VALUES
(1, 'A', CAST('2-14-20' AS date)),
(2, 'B', CAST('2-14-20' AS date)),
(3, 'C', CAST('2-15-20' AS date)),
(1, 'B', CAST('2-15-20' AS date)),
(2, 'A', CAST('2-16-20' AS date)),
(3, 'B', CAST('2-17-20' AS date)),
(1, 'D', CAST('2-18-20' AS date)))

SELECT *
FROM posts
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query

,user_id,post,time
0,1,A,2020-02-14
1,2,B,2020-02-14
2,3,C,2020-02-15
3,1,B,2020-02-15
4,2,A,2020-02-16
5,3,B,2020-02-17
6,1,D,2020-02-18


In [12]:
# A) last time each post was seen, in reverse chronological order

sql_query = """
WITH posts (user_id, post, time)
AS (VALUES
(1, 'A', CAST('2-14-20' AS date)),
(2, 'B', CAST('2-14-20' AS date)),
(3, 'C', CAST('2-15-20' AS date)),
(1, 'B', CAST('2-15-20' AS date)),
(2, 'A', CAST('2-16-20' AS date)),
(3, 'B', CAST('2-17-20' AS date)),
(1, 'D', CAST('2-18-20' AS date)))

SELECT post,
       MAX(time) AS last_post
FROM posts
GROUP BY post
ORDER BY last_post DESC
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query


,post,last_post
0,D,2020-02-18
1,B,2020-02-17
2,A,2020-02-16
3,C,2020-02-15


In [17]:
# B) last post each user has seen, in chronological order

sql_query = """
WITH posts (user_id, post, time)
AS (VALUES
(1, 'A', CAST('2-14-20' AS date)),
(2, 'B', CAST('2-14-20' AS date)),
(3, 'C', CAST('2-15-20' AS date)),
(1, 'B', CAST('2-15-20' AS date)),
(2, 'A', CAST('2-16-20' AS date)),
(3, 'B', CAST('2-17-20' AS date)),
(1, 'D', CAST('2-18-20' AS date)))

-- | user | post | date |

SELECT user_id, post,
       MAX(time) AS last_post
FROM posts
GROUP BY user_id, post
ORDER BY last_post DESC
"""
df_query = pd.read_sql_query(sql_query,con)    
df_query



,user_id,post,last_post
0,1,D,2020-02-18
1,3,B,2020-02-17
2,2,A,2020-02-16
3,3,C,2020-02-15
4,1,B,2020-02-15
5,2,B,2020-02-14
6,1,A,2020-02-14


In [19]:
# B) last post each user has seen, in chronological order

sql_query = """
WITH posts (user_id, post, time)
AS (VALUES
(1, 'A', CAST('2-14-20' AS date)),
(2, 'B', CAST('2-14-20' AS date)),
(3, 'C', CAST('2-15-20' AS date)),
(1, 'B', CAST('2-15-20' AS date)),
(2, 'A', CAST('2-16-20' AS date)),
(3, 'B', CAST('2-17-20' AS date)),
(1, 'D', CAST('2-18-20' AS date)))

-- | user | post | date |

SELECT *
FROM
    (SELECT *,
           RANK() OVER(PARTITION BY user_id ORDER BY time DESC ) AS rank
    FROM posts) t
WHERE t.rank=1

"""
df_query = pd.read_sql_query(sql_query,con)    
df_query



,user_id,post,time,rank
0,1,D,2020-02-18,1
1,2,A,2020-02-16,1
2,3,B,2020-02-17,1


In [28]:
# Testing Mike's
sql_query = """
WITH posts (user_id, post, time) 
AS (VALUES(1, 'A', CAST('2-14-20' AS date)),
          (2, 'B', CAST('2-14-20' AS date)),
          (3, 'C', CAST('2-15-20' AS date)),
          (1, 'B', CAST('2-15-20' AS date)),
          (2, 'A', CAST('2-16-20' AS date)),
          (3, 'B', CAST('2-17-20' AS date)),
          (1, 'D', CAST('2-18-20' AS date))),

t1 AS (
SELECT *, rank() OVER (PARTITION by user_id ORDER BY TIME desc) AS time_rank
FROM posts)

SELECT user_id, post, time_rank
FROM t1
WHERE time_rank = 1
ORDER BY 3


"""
df_query = pd.read_sql_query(sql_query,con)    
df_query


,user_id,post,time_rank
0,1,D,1
1,2,A,1
2,3,B,1


In [21]:
# B) last post each user has seen, in chronological order

# Kongrath

sql_query = """
WITH posts (user_id, post, time)
AS (VALUES
(1, 'A', CAST('2-14-20' AS date)),
(2, 'B', CAST('2-14-20' AS date)),
(3, 'C', CAST('2-15-20' AS date)),
(1, 'B', CAST('2-15-20' AS date)),
(2, 'A', CAST('2-16-20' AS date)),
(3, 'B', CAST('2-17-20' AS date)),
(1, 'D', CAST('2-18-20' AS date)))

-- | user | post | date |

SELECT temp.User_id, Posts.Post
FROM 
	(SELECT User_id, MAX(Time) AS maxT
	FROM Posts
	GROUP BY User_id) AS temp
	
	JOIN Posts
    ON temp.User_id = Posts.User_id AND temp.maxT = Posts.Time
ORDER BY temp.maxT


"""
df_query = pd.read_sql_query(sql_query,con)    
df_query



,user_id,post
0,2,A
1,3,B
2,1,D


# AR/LW problem

Input: Table with transaction date, seller id, buyer id, quantity, price, etc.

Output: Return the top 3 sellers with the highest revenue each day

| date |  seller_id | 

|2020-05-06  | 015  | 
|2020-05-06  | 021  |
|2020-05-06  | 003  |

|2020-05-05  | 019  |
|2020-05-05  | 011  |
|2020-05-05  | 002  |  


## First attempt (~15 min)

In [ ]:
-- outer/fourth table is what I did last 
SELECT t3.td,   
       t3.sid,
       t3.total_revenue,
       t3.rank
FROM
-- get rank  (did this third)             
(SELECT t2.td,
       t2.sid,
       t2.total_revenue,
       RANK() OVER(PARTITION BY t2.date, ORDER BY t2.total_revenue DESC) AS rank
FROM
-- get total revenue for each date, seller  (did this second)
(SELECT t1.td,
        t1.sid,
       SUM(revenue) AS total_revenue
FROM
   -- get revenue for each transaction (did this first) -- can combine with second
   (SELECT transaction_date td,
           seller_id sid,
           (price*quantity) AS revenue
    FROM input_table) AS t1
GROUP BY t1.td, t1.sid) AS t2) AS t3

WHERE t3.rank <= 3



## Testing with custom table

transaction date, seller id, buyer id, quantity, price

In [208]:
my_date_list = generate_dates(10, "2020-01-01 13:40:00", "2020-01-04 14:50:00")
my_seller_ids = generate_custom_vals(generate_codes(4), 10)   # Duplicated number of seller_ids
my_buyer_ids = generate_custom_vals(generate_codes(3), 10)   # Duplicated number of buyer_ids
# Custom quantity
my_list2consider = [1,2,3,5,10]
quantity =  generate_custom_vals(my_list2consider, 10)
# Custom price
price = generate_values(10, 1, 20)

In [212]:
col_1 = pd.to_datetime(my_date_list)
col_2 = my_seller_ids
col_3 = my_buyer_ids
col_4 = quantity
col_5 = price
table1 = pd.DataFrame([col_1, col_2, col_3, col_4, col_5]).T
table1.columns = ['td', 'seller_id', 'buyer_id', 'quantity', 'price']

table1

# Temp table created here that I'll just over-write with each new problem
table1.to_sql('input_table', engine, if_exists='replace')

In [217]:
# Test first attempt

sql_query = """

-- outer/fourth table is what I did last 
SELECT t3.td,   
       t3.sid,
       t3.total_revenue,
       t3.rank
FROM
-- get rank  (did this third)             
(SELECT t2.td,
       t2.sid,
       t2.total_revenue,
       RANK() OVER(PARTITION BY t2.td ORDER BY t2.total_revenue DESC) AS rank
FROM
-- get total revenue for each date, seller  (did this second)
(SELECT t1.td,
        t1.sid,
       SUM(revenue) AS total_revenue
FROM
   -- get revenue for each transaction (did this first) -- can combine with second
   (SELECT td,
           seller_id sid,
           (price*quantity) AS revenue
    FROM input_table) AS t1
GROUP BY t1.td, t1.sid) AS t2) AS t3

WHERE t3.rank <= 3

"""
df_query = pd.read_sql_query(sql_query,con)    
df_query



,td,sid,total_revenue,rank
0,2020-01-01 18:01:57,133,22.0,1
1,2020-01-02 07:38:53,341,13.0,1
2,2020-01-02 09:32:45,341,10.0,1
3,2020-01-02 13:48:33,625,28.0,1
4,2020-01-03 09:58:08,341,100.0,1
5,2020-01-03 11:07:12,341,50.0,1
6,2020-01-03 11:22:48,341,32.0,1
7,2020-01-04 02:33:28,455,51.0,1
8,2020-01-04 09:49:35,625,19.0,1
9,2020-01-04 11:31:49,341,90.0,1


In [221]:
# Figure out date

sql_query = """

   SELECT DATE(td),
           seller_id sid,
           (price*quantity) AS revenue
    FROM input_table

"""
df_query = pd.read_sql_query(sql_query,con)    
df_query



,date,sid,revenue
0,2020-01-01,133,22
1,2020-01-02,341,13
2,2020-01-02,341,10
3,2020-01-02,625,28
4,2020-01-03,341,100
5,2020-01-03,341,50
6,2020-01-03,341,32
7,2020-01-04,455,51
8,2020-01-04,625,19
9,2020-01-04,341,90


In [224]:
# Test first attempt

sql_query = """

-- outer/fourth table is what I did last 
SELECT t3.date,   
       t3.sid,
       t3.total_revenue,
       t3.rank
FROM
-- get rank  (did this third)             
(SELECT t2.date,
       t2.sid,
       t2.total_revenue,
       RANK() OVER(PARTITION BY t2.date ORDER BY t2.total_revenue DESC) AS rank
FROM
-- get total revenue for each date, seller  (did this second)
(SELECT t1.date,
        t1.sid,
       SUM(revenue) AS total_revenue
FROM
   -- get revenue for each transaction (did this first) -- can combine with second
   (SELECT DATE(td) AS date,   -- extract date from date/timestamp
           seller_id sid,
           (price*quantity) AS revenue
    FROM input_table) AS t1
GROUP BY t1.date, t1.sid) AS t2) AS t3

WHERE t3.rank <= 3

"""
df_query = pd.read_sql_query(sql_query,con)    
df_query



,date,sid,total_revenue,rank
0,2020-01-01,133,22.0,1
1,2020-01-02,625,28.0,1
2,2020-01-02,341,23.0,2
3,2020-01-03,341,182.0,1
4,2020-01-04,341,90.0,1
5,2020-01-04,455,51.0,2
6,2020-01-04,625,19.0,3


In [231]:
# Test first attempt

sql_query = """

SELECT DATE(t1.td) AS date,
       t1.seller_id sid,
       SUM(price*quantity) AS total_revenue
FROM input_table t1
GROUP BY date, sid

"""
df_query = pd.read_sql_query(sql_query,con)    
df_query



,date,sid,total_revenue
0,2020-01-03,341,182.0
1,2020-01-04,625,19.0
2,2020-01-02,341,23.0
3,2020-01-04,455,51.0
4,2020-01-02,625,28.0
5,2020-01-01,133,22.0
6,2020-01-04,341,90.0


In [226]:
# Test first attempt

sql_query = """

-- outer/fourth table is what I did last 
SELECT t3.date,   
       t3.sid,
       t3.total_revenue,
       t3.rank
FROM
-- get rank  (did this third)             
(SELECT t2.date,
       t2.sid,
       t2.total_revenue,
       RANK() OVER(PARTITION BY t2.date ORDER BY t2.total_revenue DESC) AS rank
FROM
-- get total revenue for each date, seller  (did this second)
(SELECT DATE(t1.td) AS date,
        t1.sid,
       SUM(price*quantity) AS total_revenue
FROM input_table t1
GROUP BY t1.date, t1.sid) AS t2) 

WHERE t2.rank <= 3

"""
df_query = pd.read_sql_query(sql_query,con)    
df_query



DatabaseError: Execution failed on sql '

-- outer/fourth table is what I did last 
SELECT t3.date,   
       t3.sid,
       t3.total_revenue,
       t3.rank
FROM
-- get rank  (did this third)             
(SELECT t2.date,
       t2.sid,
       t2.total_revenue,
       RANK() OVER(PARTITION BY t2.date ORDER BY t2.total_revenue DESC) AS rank
FROM
-- get total revenue for each date, seller  (did this second)
(SELECT DATE(t1.td) AS date,
        t1.sid,
       SUM(price*quantity) AS total_revenue
FROM input_table AS t1
GROUP BY t1.date, t1.sid) AS t2) AS t3

WHERE t3.rank <= 3

': column t1.sid does not exist
LINE 17:         t1.sid,
                 ^


In [238]:
# Lufan's attempt

sql_query = """

WITH temp AS
    (SELECT seller_id,
    DATE(td) AS date,
    SUM(price*quantity) AS revenue,
    RANK() OVER(PARTITION BY DATE(td) ORDER BY SUM(price*quantity) DESC) AS rank
    FROM input_table
    GROUP BY seller_id, date)
        
SELECT *
FROM temp
WHERE rank <= 3

"""
df_query = pd.read_sql_query(sql_query,con)    
df_query


,seller_id,date,revenue,rank
0,133,2020-01-01,22.0,1
1,625,2020-01-02,28.0,1
2,341,2020-01-02,23.0,2
3,341,2020-01-03,182.0,1
4,341,2020-01-04,90.0,1
5,455,2020-01-04,51.0,2
6,625,2020-01-04,19.0,3


In [240]:
# Lufan's attempt

sql_query = """

WITH temp AS
    (SELECT seller_id,
    DATE(td) AS date,
    SUM(price*quantity) AS revenue,
    RANK() OVER(PARTITION BY DATE(td) ORDER BY SUM(price*quantity) DESC) AS rank
    FROM input_table
    GROUP BY date, seller_id)
        
SELECT *
FROM temp


"""
df_query = pd.read_sql_query(sql_query,con)    
df_query


,seller_id,date,revenue,rank
0,133,2020-01-01,22.0,1
1,625,2020-01-02,28.0,1
2,341,2020-01-02,23.0,2
3,341,2020-01-03,182.0,1
4,341,2020-01-04,90.0,1
5,455,2020-01-04,51.0,2
6,625,2020-01-04,19.0,3


# --